In [ ]:
from datetime import datetime
import os
import requests
import zipfile
from bs4 import BeautifulSoup

def download_file_from_google_colab(output_file_path):
    from google.colab import files
    files.download(output_file_path)
    print(f"File downloaded: {output_file_path}")

session_cookie = 'sessionid=sv928zw4xkpipdd82btdegbqyruzysf8'
RFNSA_No = '2323012'
# Send a GET request to the URL
response = session.get('https://www.rfnsa.com.au/'+ RFNSA_No +'/documents', headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                        'Cookie': session_cookie})

In [ ]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.text, "html.parser")
    projects = soup.find_all("tr", class_="test-project-doc")

    for x in range(0, len(projects))
        project_date = projects[p].find_all("td")[0].get_text(strip=True)
        carrier = projects[p].find_all("td")[1].get_text(strip=True)
        project_name = projects[p].find_all("td")[2].get_text(strip=True)
        System = projects[p].find_all("td")[3].get_text(strip=True)
        id_documents = projects[p].find_all("td")[4].find('a')['data-target'].replace('#','')
        documents = soup.find("div", {"id": id_documents}).find_all("tr", class_="test-project-inner-doc clickable-row")

        for d in range(0,len(documents)):
            document_name = documents[d].find_all('td')[0].get_text(separator = '\n', strip = True).split('\n')[1]
            upload_date = datetime.strptime(documents[d].find_all('td')[2].text, '%d %b %Y %H:%M:%S').strftime('%Y-%m-%d')
            document_url = 'https://www.rfnsa.com.au'+ documents[d].find_all('a',class_="download-document clickable-row-anchor clickable-row-no-spinner")[0]['data-url']
            file_name =  f"{carrier}_{project_name}_{upload_date.replace(' ', '_')}_{document_name}"

            document_response = session.get(document_url, headers={'User-Agent':'Mozilla/5.0 (Linux; Android 12; SM-S906N Build/QP1A.190711.020; wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/80.0.3987.119 Mobile Safari/537.36',
                            'Cookie': session_cookie})  
            if document_response.status_code == 200:
                # Check if the document is a zip file
                if document_response.headers.get("Content-Type") == "application/zip":
                    zip_file_path = "temp.zip"
                    with open(zip_file_path, "wb") as zip_file:
                        zip_file.write(document_response.content)

                    # Unzip the file
                    with zipfile.ZipFile(zip_file_path, "r") as zip_ref:
                        # Extract and save the files with the new names
                        for zip_info in zip_ref.infolist():
                            extracted_file_name = f"{carrier}_{project_name}_{upload_date.replace(' ', '_')}_{zip_info.filename}"
                            zip_ref.extract(zip_info, path=extracted_file_name)
                            if 'google.colab' in str(get_ipython()):
                                download_file_from_google_colab(extracted_file_name)

                    # Remove the original zip file
                    os.remove(zip_file_path)

                    print(f"Unzipped and copied to roof folder: {file_name}")
                else:
                    # Save the document with the new name
                    with open(file_name, "wb") as file:
                        file.write(document_response.content)
                        if 'google.colab' in str(get_ipython()):
                            download_file_from_google_colab(file_name)
                    print(f"Downloaded: {file_name}")
            else:
                print(f"Failed to download: {document_url}")
